<div style="background: linear-gradient(135deg,rgb(65, 89, 195) 0%,rgb(98, 120, 143) 100%); padding: 30px; border-radius: 15px; margin: 20px 0; text-align: center; color: white; box-shadow: 0 8px 32px rgba(111, 89, 89, 0.1);">
    <h1 style="font-size: 2.8em; margin: 0 0 15px 0; font-weight: 700; text-shadow: 2px 2px 4px rgba(93, 78, 78, 0.3);">Notebook 1: Data Collection & Wrangling</h1>
    <h2 style="font-size: 1.6em; margin: 0; font-weight: 400; opacity: 0.9;">SpaceX Falcon 9 First Stage Landing Prediction</h2>
</div>

<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="400" alt="Skills Network Logo">
    </a>
</p>

# **SpaceX  Falcon 9 first stage Landing Prediction**
# Lab 1: Collecting the data

<strong><p>In this capstone, we will predict if the Falcon 9 first stage will land successfully. SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars; other providers cost upward of 165 million dollars each, much of the savings is because SpaceX can reuse the first stage. Therefore if we can determine if the first stage will land, we can determine the cost of a launch. This information can be used if an alternate company wants to bid against SpaceX for a rocket launch. In this lab, you will collect and make sure the data is in the correct format from an API. The following is an example of a successful and launch.<p></strong>


In [ ]:
# Requests allows us to make HTTP requests to get data from an API
import requests
# Pandas is a powerful library for data manipulation and analysis
import pandas as pd
# NumPy is used for numerical operations
import numpy as np
# Datetime helps us handle date objects
import datetime
# BeautifulSoup is for web scraping to parse HTML content
from bs4 import BeautifulSoup
import re
import unicodedata
import os

# Setting pandas display options for better viewing
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [13]:
#Global variables 
BoosterVersion = []
PayloadMass = []
Orbit = []
LaunchSite = []
Outcome = []
Flights = []
GridFins = []
Reused = []
Legs = []
LandingPad = []
Block = []
ReusedCount = []
Serial = []
Longitude = []
Latitude = []
# Takes the dataset and uses the rocket column to call the API and append the data to the list
def getBoosterVersion(data):
    for x in data['rocket']:
       if x:
        response = requests.get("https://api.spacexdata.com/v4/rockets/"+str(x)).json()
        BoosterVersion.append(response['name'])

# Takes the dataset and uses the launchpad column to call the API and append the data to the list
def getLaunchSite(data):
    for x in data['launchpad']:
       if x:
         response = requests.get("https://api.spacexdata.com/v4/launchpads/"+str(x)).json()
         Longitude.append(response['longitude'])
         Latitude.append(response['latitude'])
         LaunchSite.append(response['name'])
        
# Takes the dataset and uses the payloads column to call the API and append the data to the lists
def getPayloadData(data):
    for load in data['payloads']:
       if load:
        response = requests.get("https://api.spacexdata.com/v4/payloads/"+load).json()
        PayloadMass.append(response['mass_kg'])
        Orbit.append(response['orbit'])
            
# Takes the dataset and uses the cores column to call the API and append the data to the lists
def getCoreData(data):
    for core in data['cores']:
            if core['core'] != None:
                response = requests.get("https://api.spacexdata.com/v4/cores/"+core['core']).json()
                Block.append(response['block'])
                ReusedCount.append(response['reuse_count'])
                Serial.append(response['serial'])
            else:
                Block.append(None)
                ReusedCount.append(None)
                Serial.append(None)
            Outcome.append(str(core['landing_success'])+' '+str(core['landing_type']))
            Flights.append(core['flight'])
            GridFins.append(core['gridfins'])
            Reused.append(core['reused'])
            Legs.append(core['legs'])
            LandingPad.append(core['landpad'])

spacex_url="https://api.spacexdata.com/v4/launches/past"
# response = requests.get(spacex_url)
# response.raise_for_status
static_json_url='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/API_call_spacex_api.json'
response = requests.get(static_json_url)
response.raise_for_status()
if response.status_code == 200:
    data = pd.json_normalize(response.json())
    print(data)
else:
    print(f"Error: {response.status_code}")
data = data[['rocket', 'payloads', 'launchpad', 'cores', 'flight_number', 'date_utc']]
data = data[data['cores'].map(len)==1]
data = data[data['payloads'].map(len)==1]
data['cores'] = data['cores'].map(lambda x: x[0])
data['payloads'] = data['payloads'].map(lambda x: x[0])
data['date'] = pd.to_datetime(data['date_utc']).dt.date
data = data[data['date'] <= datetime.date(2020,11,13)]
# Call getBoosterVersion
getBoosterVersion(data)
BoosterVersion[0:5]
# Call getLaunchSite
getLaunchSite(data)
# Call getPayloadData
getPayloadData(data)
# Call getCoreData
getCoreData(data)

         static_fire_date_utc  static_fire_date_unix    tbd    net  window  \
0    2006-03-17T00:00:00.000Z           1.142554e+09  False  False     0.0   
1                        None                    NaN  False  False     0.0   
2                        None                    NaN  False  False     0.0   
3    2008-09-20T00:00:00.000Z           1.221869e+09  False  False     0.0   
4                        None                    NaN  False  False     0.0   
..                        ...                    ...    ...    ...     ...   
102                      None                    NaN  False  False     0.0   
103  2020-10-17T05:23:00.000Z           1.602912e+09  False  False     NaN   
104  2020-10-21T12:55:00.000Z           1.603285e+09  False  False     NaN   
105  2020-09-25T05:42:00.000Z           1.601013e+09  False  False     NaN   
106  2020-11-11T16:17:00.000Z           1.605111e+09  False  False     NaN   

                       rocket  success  \
0    5e9d0d95eda69955

In [14]:
launch_dict = {'FlightNumber': list(data['flight_number']),
'Date': list(data['date']),
'BoosterVersion':BoosterVersion,
'PayloadMass':PayloadMass,
'Orbit':Orbit,
'LaunchSite':LaunchSite,
'Outcome':Outcome,
'Flights':Flights,
'GridFins':GridFins,
'Reused':Reused,
'Legs':Legs,
'LandingPad':LandingPad,
'Block':Block,
'ReusedCount':ReusedCount,
'Serial':Serial,
'Longitude': Longitude,
'Latitude': Latitude}

launch_dict_df = pd.DataFrame(launch_dict)
display(launch_dict_df.describe())
data_falcon9 = launch_dict_df[launch_dict_df['BoosterVersion'] == 'Falcon 9']
data_falcon9.loc[:,'FlightNumber'] = list(range(1,len(data_falcon9)+1))
data_falcon9.head()

,FlightNumber,PayloadMass,Flights,Block,ReusedCount,Longitude,Latitude
count,94.000000,88.000000,94.000000,90.000000,94.000000,94.000000,94.000000
mean,54.202128,5919.165341,1.755319,3.500000,3.053191,-75.553302,28.581782
std,30.589048,4909.689575,1.197544,1.595288,4.153938,53.391880,4.639981
min,1.000000,20.000000,1.000000,1.000000,0.000000,-120.610829,9.047721
25%,28.250000,2406.250000,1.000000,2.000000,0.000000,-80.603956,28.561857
50%,52.500000,4414.000000,1.000000,4.000000,1.000000,-80.577366,28.561857
75%,81.500000,9543.750000,2.000000,5.000000,4.000000,-80.577366,28.608058
max,106.000000,15600.000000,6.000000,5.000000,13.000000,167.743129,34.632093


,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
4,1,2010-06-04,Falcon 9,NaN,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0003,-80.577366,28.561857
5,2,2012-05-22,Falcon 9,525.0,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0005,-80.577366,28.561857
6,3,2013-03-01,Falcon 9,677.0,ISS,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0007,-80.577366,28.561857
7,4,2013-09-29,Falcon 9,500.0,PO,VAFB SLC 4E,False Ocean,1,False,False,False,None,1.0,0,B1003,-120.610829,34.632093
8,5,2013-12-03,Falcon 9,3170.0,GTO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B1004,-80.577366,28.561857


In [17]:
# Calculate the mean value of PayloadMass column
data_falcon9['PayloadMass'] = data_falcon9.loc[:,'PayloadMass'].replace(np.nan,data_falcon9['PayloadMass'].mean())
data_falcon9.head()
# Replace the np.nan values with its mean value

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
4,1,2010-06-04,Falcon 9,6123.547647,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0003,-80.577366,28.561857
5,2,2012-05-22,Falcon 9,525.000000,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0005,-80.577366,28.561857
6,3,2013-03-01,Falcon 9,677.000000,ISS,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0007,-80.577366,28.561857
7,4,2013-09-29,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1,False,False,False,None,1.0,0,B1003,-120.610829,34.632093
8,5,2013-12-03,Falcon 9,3170.000000,GTO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B1004,-80.577366,28.561857


In [16]:
data_falcon9.isnull().sum()

FlightNumber       0
Date               0
BoosterVersion     0
PayloadMass        0
Orbit              0
LaunchSite         0
Outcome            0
Flights            0
GridFins           0
Reused             0
Legs               0
LandingPad        26
Block              0
ReusedCount        0
Serial             0
Longitude          0
Latitude           0
dtype: int64

In [20]:
data_falcon9.to_csv(os.getcwd()+'/data_falcon9.csv',index=False)

# **Space X  Falcon 9 First Stage Landing Prediction**
Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia
In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


In [21]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"
# use requests.get() method with the provided static_url
# assign the response to a object
# First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.
response = requests.get(static_url)
print(f"HTTP Status Code: {response.status_code}")
print(f"Response Headers: {dict(response.headers)}")
print(f"Content Type: {response.headers.get('content-type')}")
print(f"Response URL: {response.url}")
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
soup = BeautifulSoup(response.text, 'html.parser')
# Use the find_all() method to find all tables in the HTML page
html_tables = list(soup.find_all('table'))
# Print the number of tables found
print(f"Number of tables found: {len(html_tables)}")
# Print the first table
print(html_tables[0])
# Print the second table
print(html_tables[1])
# Print the third table

HTTP Status Code: 200
Response Headers: {'date': 'Sun, 27 Jul 2025 10:58:22 GMT', 'server': 'mw-web.eqiad.main-6b9dc868f9-z8htq', 'x-content-type-options': 'nosniff', 'content-language': 'en', 'accept-ch': '', 'last-modified': 'Sun, 27 Jul 2025 06:14:06 GMT', 'content-type': 'text/html; charset=UTF-8', 'content-encoding': 'gzip', 'age': '2', 'accept-ranges': 'bytes', 'vary': 'Accept-Encoding,X-Subdomain,Cookie,Authorization', 'x-cache': 'cp1104 miss, cp1104 miss', 'x-cache-status': 'miss', 'server-timing': 'cache;desc="miss", host;desc="cp1104"', 'strict-transport-security': 'max-age=106384710; includeSubDomains; preload', 'report-to': '{ "group": "wm_nel", "max_age": 604800, "endpoints": [{ "url": "https://intake-logging.wikimedia.org/v1/events?stream=w3c.reportingapi.network_error&schema_uri=/w3c/reportingapi/network_error/1.0.0" }] }', 'nel': '{ "report_to": "wm_nel", "max_age": 604800, "failure_fraction": 0.05, "success_fraction": 0.0}', 'set-cookie': 'WMF-Last-Access=27-Jul-2025;P

In [23]:

def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


column_names = []

# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names
first_launch_table = list(soup.find_all(['table']))[2]
all_th_elements = first_launch_table.find_all('th')
for th in all_th_elements:
    name = extract_column_from_header(th)
    if name and len(name) > 0:
        column_names.append(name)

print(column_names)

## TASK 3: Create a data frame by parsing the launch HTML tables
# We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe

launch_dict = dict.fromkeys(column_names)

# Initialize all dictionary values as empty lists instead of None
for key in launch_dict:
    launch_dict[key] = []

# Remove the problematic key and add the required keys
if 'Date and time ( )' in launch_dict:
    del launch_dict['Date and time ( )']

# Ensure all required keys exist
required_keys = ['Flight No.', 'Launch Site', 'Payload', 'Payload Mass', 'Orbit', 'Customer', 'Launch outcome', 'Version Booster', 'Booster landing', 'Date', 'Time']
for key in required_keys:
    if key not in launch_dict:
        launch_dict[key] = []
        
        
# FIRST EXTRACTION - Creates a fresh launch_dict and extracts data once
extracted_row = 0
for table_number, table in enumerate(soup.find_all('table', {"class": "wikitable plainrowheaders collapsible"})):
    for rows in table.find_all('tr'):
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()
        else:
            flag=False
        row = rows.find_all('td')
        if flag:
            extracted_row += 1
            # Append to lists instead of overwriting
            launch_dict['Flight No.'].append(flight_number)
            datatimelist = date_time(row[0])
            launch_dict['Date'].append(datatimelist[0].strip(','))
            launch_dict['Time'].append(datatimelist[1].strip(','))
            bv = booster_version(row[1])
            if not (bv):
                bv = row[1].a.string if row[1].a else ""
            # print(bv)
            launch_dict['Version Booster'].append(bv)
            launch_site = row[2].a.string if row[2].a else ""
            launch_dict['Launch Site'].append(launch_site)
            payload = row[3].a.string if row[3].a else ""
            launch_dict['Payload'].append(payload)
            payload_mass = get_mass(row[4])
            launch_dict['Payload Mass'].append(payload_mass)
            orbit = row[5].a.string if row[5].a else ""
            launch_dict['Orbit'].append(orbit)
            if row[6].a:
                customer = row[6].a.string
            else:
                customer = row[6].get_text(strip=True)
            launch_dict['Customer'].append(customer)
            launch_outcome = list(row[7].strings)[0]
            launch_dict['Launch outcome'].append(launch_outcome)
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)

print(f"Total extracted rows: {extracted_row}")
print(f"Length of Flight No. list: {len(launch_dict['Flight No.'])}")

# Fill empty columns with NaN values
for key in launch_dict.keys():
    if not launch_dict[key]:
        launch_dict[key] = [pd.NA] * len(launch_dict['Flight No.'])

df = pd.DataFrame({
                   key:pd.Series(value) for key, value in launch_dict.items()
                   })
# or pd.DataFrame(launch_dict)
print(df.shape)
display(df.head())

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']
Total extracted rows: 121
Length of Flight No. list: 121
(121, 13)


,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Launch Site,Payload Mass,Version Booster,Booster landing,Date,Time
0,1,<NA>,Dragon Spacecraft Qualification Unit,<NA>,LEO,SpaceX,Success\n,CCAFS,0,F9 v1.07B0003.18,Failure,4 June 2010,18:45
1,2,<NA>,Dragon,<NA>,LEO,NASA,Success,CCAFS,0,F9 v1.07B0004.18,Failure,8 December 2010,15:43
2,3,<NA>,Dragon,<NA>,LEO,NASA,Success,CCAFS,525 kg,F9 v1.07B0005.18,No attempt\n,22 May 2012,07:44
3,4,<NA>,SpaceX CRS-1,<NA>,LEO,NASA,Success\n,CCAFS,"4,700 kg",F9 v1.07B0006.18,No attempt,8 October 2012,00:35
4,5,<NA>,SpaceX CRS-2,<NA>,LEO,NASA,Success\n,CCAFS,"4,877 kg",F9 v1.07B0007.18,No attempt\n,1 March 2013,15:10


# **Space X  Falcon 9 First Stage Landing Prediction**
 ## Lab 2: Data wrangling 
In this lab, we will perform some Exploratory Data Analysis (EDA) to find some patterns in the data and determine what would be the label for training supervised models. 

In the data set, there are several different cases where the booster did not land successfully. Sometimes a landing was attempted but failed due to an accident; for example, <code>True Ocean</code> means the mission outcome was successfully  landed to a specific region of the ocean while <code>False Ocean</code> means the mission outcome was unsuccessfully landed to a specific region of the ocean. <code>True RTLS</code> means the mission outcome was successfully  landed to a ground pad <code>False RTLS</code> means the mission outcome was unsuccessfully landed to a ground pad.<code>True ASDS</code> means the mission outcome was successfully landed on  a drone ship <code>False ASDS</code> means the mission outcome was unsuccessfully landed on a drone ship. 

In this lab we will mainly convert those outcomes into Training Labels with `1` means the booster successfully landed `0` means it was unsuccessful.


## Objectives
Perform exploratory  Data Analysis and determine Training Labels 

- Exploratory Data Analysis
- Determine Training Labels 


In [24]:
df=pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/dataset_part_1.csv")
df.head(10)

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,1,2010-06-04,Falcon 9,6104.959412,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857
1,2,2012-05-22,Falcon 9,525.000000,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0005,-80.577366,28.561857
2,3,2013-03-01,Falcon 9,677.000000,ISS,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0007,-80.577366,28.561857
3,4,2013-09-29,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1.0,0,B1003,-120.610829,34.632093
4,5,2013-12-03,Falcon 9,3170.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1004,-80.577366,28.561857
5,6,2014-01-06,Falcon 9,3325.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1005,-80.577366,28.561857
6,7,2014-04-18,Falcon 9,2296.000000,ISS,CCAFS SLC 40,True Ocean,1,False,False,True,NaN,1.0,0,B1006,-80.577366,28.561857
7,8,2014-07-14,Falcon 9,1316.000000,LEO,CCAFS SLC 40,True Ocean,1,False,False,True,NaN,1.0,0,B1007,-80.577366,28.561857
8,9,2014-08-05,Falcon 9,4535.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1008,-80.577366,28.561857
9,10,2014-09-07,Falcon 9,4428.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1011,-80.577366,28.561857


In [25]:
df.value_counts(subset=['LaunchSite'])

LaunchSite  
CCAFS SLC 40    55
KSC LC 39A      22
VAFB SLC 4E     13
Name: count, dtype: int64



* <b>LEO</b>: Low Earth orbit (LEO)is an Earth-centred orbit with an altitude of 2,000 km (1,200 mi) or less (approximately one-third of the radius of Earth),[1] or with at least 11.25 periods per day (an orbital period of 128 minutes or less) and an eccentricity less than 0.25.[2] Most of the manmade objects in outer space are in LEO <a href='https://en.wikipedia.org/wiki/Low_Earth_orbit'>[1]</a>.

* <b>VLEO</b>: Very Low Earth Orbits (VLEO) can be defined as the orbits with a mean altitude below 450 km. Operating in these orbits can provide a number of benefits to Earth observation spacecraft as the spacecraft operates closer to the observation<a href='https://www.researchgate.net/publication/271499606_Very_Low_Earth_Orbit_mission_concepts_for_Earth_Observation_Benefits_and_challenges'>[2]</a>.


* <b>GTO</b> A geosynchronous orbit is a high Earth orbit that allows satellites to match Earth's rotation. Located at 22,236 miles (35,786 kilometers) above Earth's equator, this position is a valuable spot for monitoring weather, communications and surveillance. Because the satellite orbits at the same speed that the Earth is turning, the satellite seems to stay in place over a single longitude, though it may drift north to south,” NASA wrote on its Earth Observatory website <a  href="https://www.space.com/29222-geosynchronous-orbit.html" >[3] </a>.


* <b>SSO (or SO)</b>: It is a Sun-synchronous orbit  also called a heliosynchronous orbit is a nearly polar orbit around a planet, in which the satellite passes over any given point of the planet's surface at the same local mean solar time <a href="https://en.wikipedia.org/wiki/Sun-synchronous_orbit">[4]</a>.
    
    
    
* <b>ES-L1</b>:At the Lagrange points the gravitational forces of the two large bodies cancel out in such a way that a small object placed in orbit there is in equilibrium relative to the center of mass of the large bodies. L1 is one such point between the sun and the earth <a href="https://en.wikipedia.org/wiki/Lagrange_point#L1_point">[5]</a> .
    
    
* <b>HEO</b> A highly elliptical orbit, is an elliptic orbit with high eccentricity, usually referring to one around Earth <a href="https://en.wikipedia.org/wiki/Highly_elliptical_orbit">[6]</a>.


* <b> ISS </b> A modular space station (habitable artificial satellite) in low Earth orbit. It is a multinational collaborative project between five participating space agencies: NASA (United States), Roscosmos (Russia), JAXA (Japan), ESA (Europe), and CSA (Canada)<a href="https://en.wikipedia.org/wiki/International_Space_Station"> [7] </a>


* <b> MEO </b> Geocentric orbits ranging in altitude from 2,000 km (1,200 mi) to just below geosynchronous orbit at 35,786 kilometers (22,236 mi). Also known as an intermediate circular orbit. These are "most commonly at 20,200 kilometers (12,600 mi), or 20,650 kilometers (12,830 mi), with an orbital period of 12 hours <a href="https://en.wikipedia.org/wiki/List_of_orbits"> [8] </a>


* <b> HEO </b> Geocentric orbits above the altitude of geosynchronous orbit (35,786 km or 22,236 mi) <a href="https://en.wikipedia.org/wiki/List_of_orbits"> [9] </a>


* <b> GEO </b> It is a circular geosynchronous orbit 35,786 kilometres (22,236 miles) above Earth's equator and following the direction of Earth's rotation <a href="https://en.wikipedia.org/wiki/Geostationary_orbit"> [10] </a>


* <b> PO </b> It is one type of satellites in which a satellite passes above or nearly above both poles of the body being orbited (usually a planet such as the Earth <a href="https://en.wikipedia.org/wiki/Polar_orbit"> [11] </a>

some are shown in the following plot:


In [ ]:
df.value_counts(subset=['Orbit'])

Orbit
GTO      27
ISS      21
VLEO     14
PO        9
LEO       7
SSO       5
MEO       3
GEO       1
ES-L1     1
HEO       1
SO        1
Name: count, dtype: int64

In [ ]:
dict.fromkeys(df['Outcome'],0)

{'None None': 0,
 'False Ocean': 0,
 'True Ocean': 0,
 'False ASDS': 0,
 'None ASDS': 0,
 'True RTLS': 0,
 'True ASDS': 0,
 'False RTLS': 0}

<code>True Ocean</code> means the mission outcome was successfully  landed to a specific region of the ocean while <code>False Ocean</code> means the mission outcome was unsuccessfully landed to a specific region of the ocean. <code>True RTLS</code> means the mission outcome was successfully  landed to a ground pad <code>False RTLS</code> means the mission outcome was unsuccessfully landed to a ground pad.<code>True ASDS</code> means the mission outcome was successfully  landed to a drone ship <code>False ASDS</code> means the mission outcome was unsuccessfully landed to a drone ship. <code>None ASDS</code> and <code>None None</code> these represent a failure to land.


In [30]:
landing_outcomes = dict(df.value_counts(subset=['Outcome']))
landing_outcomes

{('True ASDS',): np.int64(41),
 ('None None',): np.int64(19),
 ('True RTLS',): np.int64(14),
 ('False ASDS',): np.int64(6),
 ('True Ocean',): np.int64(5),
 ('False Ocean',): np.int64(2),
 ('None ASDS',): np.int64(2),
 ('False RTLS',): np.int64(1)}

In [31]:
for i,outcome in enumerate(landing_outcomes.keys()):
    print(i,outcome)

0 ('True ASDS',)
1 ('None None',)
2 ('True RTLS',)
3 ('False ASDS',)
4 ('True Ocean',)
5 ('False Ocean',)
6 ('None ASDS',)
7 ('False RTLS',)


In [47]:
bad_outcomes = {}
for i,j in landing_outcomes.items():
    print(i[0])
    if re.compile('False|None').search(i[0]):
        bad_outcomes[i[0]] = j
bad_outcomes

True ASDS
None None
True RTLS
False ASDS
True Ocean
False Ocean
None ASDS
False RTLS


{'None None': np.int64(19),
 'False ASDS': np.int64(6),
 'False Ocean': np.int64(2),
 'None ASDS': np.int64(2),
 'False RTLS': np.int64(1)}

In [48]:
landing_class = []
for outcome in df['Outcome']:
    if outcome in bad_outcomes.keys():
        landing_class.append(0)
    else:
        landing_class.append(1)
df['Class'] = landing_class
df

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude,Class
0,1,2010-06-04,Falcon 9,6104.959412,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857,0
1,2,2012-05-22,Falcon 9,525.000000,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0005,-80.577366,28.561857,0
2,3,2013-03-01,Falcon 9,677.000000,ISS,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0007,-80.577366,28.561857,0
3,4,2013-09-29,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1.0,0,B1003,-120.610829,34.632093,0
4,5,2013-12-03,Falcon 9,3170.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1004,-80.577366,28.561857,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,86,2020-09-03,Falcon 9,15400.000000,VLEO,KSC LC 39A,True ASDS,2,True,True,True,5e9e3032383ecb6bb234e7ca,5.0,2,B1060,-80.603956,28.608058,1
86,87,2020-10-06,Falcon 9,15400.000000,VLEO,KSC LC 39A,True ASDS,3,True,True,True,5e9e3032383ecb6bb234e7ca,5.0,2,B1058,-80.603956,28.608058,1
87,88,2020-10-18,Falcon 9,15400.000000,VLEO,KSC LC 39A,True ASDS,6,True,True,True,5e9e3032383ecb6bb234e7ca,5.0,5,B1051,-80.603956,28.608058,1
88,89,2020-10-24,Falcon 9,15400.000000,VLEO,CCAFS SLC 40,True ASDS,3,True,True,True,5e9e3033383ecbb9e534e7cc,5.0,2,B1060,-80.577366,28.561857,1


In [50]:
success_rate = {}
for site in df['LaunchSite'].unique():
    exact_site = df[df['LaunchSite'] == site]
    success_rate[site] = f'{round(exact_site['Class'].sum() / exact_site.shape[0],4)*100}%'
success_rate

{'CCAFS SLC 40': '60.0%',
 'VAFB SLC 4E': '76.92%',
 'KSC LC 39A': '77.27000000000001%'}

In [51]:
df.to_csv(os.path.join(os.getcwd(),"dataset_part_2.csv"), index=False)